# TensorFlow introduction

In [1]:
import tensorflow as tf

## Tensor 

Tensor is a multidimensional array (Generalization of Matrix). Can be used to create constants or variables.

Here are some example of tensors:

In [2]:
# Rank 0 tensor 
print(tf.constant(4.0))
# Rank 1 tensor with specific type
print(tf.constant([1, 2], dtype='int64'))
# Rank 3 tensor created as variable with specific name
print(tf.Variable([[[1,2],[3,4]], [[5,6], [7,8]]], name='t2'))

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(2,), dtype=int64)
<tf.Variable 't2:0' shape=(2, 2, 2) dtype=int32_ref>


## Graph

In TF we define graph of computation. Graph consist of tensors and operations.

Here is simple graph for adding 2 tensors:

In [3]:
t1 = tf.constant([1, 2])
t2 = tf.constant([3, 4])
g1 = tf.add(t1, t2, name='my_add')

As you can see we don't get result yet. We only defined graph. To execute it we need sessions.

## Session

Session is used to execute graph and get result from computation.
Here we execute our just defined graph

In [4]:
session = tf.Session()
session.run(g1)

array([4, 6], dtype=int32)

In our first graph we only used constant.
If we also want to use variables then we need to initialize them before we run computation.

Lets define graph with variables

In [5]:
t1 = tf.constant([1, 2])
t2 = tf.Variable([3, 4])
g2 = tf.multiply(t1, t2)

## Variable initialization 

Variables need to be initialized before we can use them. We do it by creating special operation which we will run in our session before any other operations.

In [6]:
# Add an Op to initialize global variables.
init_op = tf.global_variables_initializer()

# Launch the graph in a session.
with tf.Session() as sess:
    # Run the Op that initializes global variables.
    sess.run(init_op)
    # ...you can now run any Op that uses variable values...
    print(sess.run(g2))

[3 8]


## Placeholders

Placeholders allows to provided tensorf during graph executing. 

The following graph will add 2 vectors where one will be provided during session execution.

In [7]:
t1 = tf.constant([1.0, 2.0])
x = tf.placeholder(tf.float32)
g3 = tf.multiply(t1, x)

Now we can run this graph with different values

In [8]:
# We still need to initialize variables
init_op = tf.global_variables_initializer()

# Launch first session
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(g3, {x: [4, 5]}))
    print(sess.run(g3, {x: [12, 32]}))

[  4.  10.]
[ 12.  64.]


## Loss function

Loss function is a metrics which defines how good our model is.

Lets build simple linear model

In [9]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

We can evaluate this model at few points

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(linear_model, {x: [4, 5]}))

[ 0.90000004  1.20000005]


Now we define loss function. We will use sum of squares

In [11]:
y = tf.placeholder(tf.float32)
squared_delta = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_delta)

# Let's check this model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


## Optimizers

Optimizers change Variables during training, so the model during training is getting closer to the real model.

We will use Gradient Descent.

In [12]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 1000 training steps
    for i in range(1000):
        # Single step in gradient descent
        sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
    # And now check what parameters were learned. We expect W = -1, b = 1
    print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## Save and Load state

TF allows to save and load variables. Variables are saved as dictionary with variable name as a key and its value.
It means that it is possible to load all variables at once or any single variable using its name.

In [17]:
c = tf.Variable([-.3, .4, 5.], tf.float32)
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(c))
    saver.save(sess, "../models/intro.ckpt")
    # Modify variable
    sess.run(tf.assign(c, [0., 0., 0.]))
    print(sess.run(c))
    # Load variable
    saver.restore(sess, "../models/intro.ckpt")
    print(sess.run(c))

[-0.30000001  0.40000001  5.        ]
[ 0.  0.  0.]
INFO:tensorflow:Restoring parameters from ../models/intro.ckpt
[-0.30000001  0.40000001  5.        ]
